This example shows how to create GW-BSE workflow using pyGWBSE
package. We will show how to compute QP bandstructure for AlN
at the partial self-consistent (GW0) level by obtaining input 
structure from Material Project (MP) database and BSE absorption 
spectra for Silicon by using input structure from a POSCAR file.

In [1]:
#First let's load the required libraries
import sys
sys.path.append('../')
from make_wf import read_input, create_wfs
from fireworks import LaunchPad
from pymatgen.ext.matproj import MPRester
from pymatgen import Structure

In [2]:
#Now let's check the content of the 'input.yaml' file.
!cat 'input.yaml'

PARAMS:
  kpar: 8                           # KPAR to be used for VASP simulations'
  ppn: 13                           # NPROCS/KPAR ; NPROCS=number of total processors for VASP simulations  
  reciprocal_density: 100           # reciprocal density that determines the number of k-points 
  encutgw: 100                      # ENCUTGW to be used for VASP simulations
  nbgwfactor: 3                     # NBANDS=nbgwfactor*nocc' ; nocc' is the smallest multiple of 
                                    # ppn that is larger than number of occupied bands
  nomegagw: 80                      # NOMEGAGW to be used for VASP simulations
  convparam: NBANDS                 # NBANDS/ENCUTGW/NOMEGA parameter to run convergence test for  
  convsteps: 2                      # steps to be used for convergence test
  conviter: 5                       # maximum number of iteration in convergence test 
  enwinbse: 3.0                     # energy window in BSE calculations



STRUCTURE:
  source: MID     

In [3]:
"""
    'read_input' function reads input parameters from input.yaml file.
    
     Returns:
    
     structure: pymatgen structure object for the input structure
     input_dict: a dictionary of all the input paramters
     
"""

structure, input_dict = read_input()

In [4]:
"""
    'create_wfs' function creates the GW-BSE workflow based on input specification.
    
    Args:
    
     structure: pymatgen structure object for the input structure
     input_dict: a dictionary of all the input paramters
     
    Returns:
    
     wf_gwbse: GW-BSE workflow as a Fireworks workflow object.
         
"""

wf_gwbse=create_wfs(structure, input_dict)

-------------------------------------------
material:  mp-661
Information for efficient parallelization
You have  8 occupied bands
You have  48 kpoints
You have  [[9, 9, 6]] k-grid
Will perform convergence test for:  NBANDS
Values that will be used for convergence test: 39 65 91 117 143 
KPAR= 8
reciprocal_density= 100
-------------------------------------------


/home/tatha/anaconda3/envs/cms/lib/python3.6/site-packages/pymatgen/io/vasp/sets.py:447: BadInputSetWarning:

Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.



In [5]:
#Load the Fireworks Launchpad 
lpad = LaunchPad.auto_load() 

In [6]:
#Add the workflow you just created
lpad.add_wf(wf_gwbse)

2022-08-04 15:20:07,507 INFO Added a workflow. id_map: {-6: 1, -5: 2, -4: 3, -3: 4, -2: 5, -1: 6}


{-6: 1, -5: 2, -4: 3, -3: 4, -2: 5, -1: 6}

In [7]:
#Check the Fireworks loaded to the Launchpad
lp='lpad get_fws'
!{lp}

/home/tatha/anaconda3/envs/cms/lib/python3.6/site-packages/pymatgen/io/vasp/sets.py:447: BadInputSetWarning:

Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.

[
    {
        "fw_id": 1,
        "created_on": "2022-08-04T09:50:01.287246",
        "updated_on": "2022-08-04T09:50:01.287246",
        "name": "mp-661-WANNIER",
        "state": "WAITING"
    },
    {
        "fw_id": 2,
        "created_on": "2022-08-04T09:50:01.287228",
        "updated_on": "2022-08-04T09:50:01.287229",
        "name": "mp-661-GW",
        "state": "WAITING"
    },
    {
        "fw_id": 3,
        "created_on": "2022-08-04T09:50:01.287157",
        "updated_on": "2022-08-04T09:50:01.287158",
        "name": "mp-661-CONV",
        "state": "WAITING"
   

As you can see there are 6 fireworks loaded to to the Launchpad. If we don't set skip_bse to true,
there would be one more firework named 'mp-661-BSE' and all possible 7 fireworks would be there 
in the workflow. 

Now, let's create another workflow to compute absorption spectra of Silicon. We will 
provide the input structure from a POSCAR file in this example instead of getting it 
from MP database.   

In [8]:
#Let's change some of the input parameters to create the new GW-BSE workflow 

structure = Structure.from_file(filename='POSCAR') 
#We want to read the structure from POSCAR file.

input_dict["PARAMS"]["mat_name"] = 'Si_test'       
#We need to set some identifier for the database.

input_dict["PARAMS"]["nbgwfactor"] = 5             
#Let's use a large number of unoccupied bands 
#and skip the convergence test.
    
input_dict["PARAMS"]["reciprocal_density"] = 250   
#Let's use a denser k-mesh so that we get accurate 
#BSE absorption spectra.
    
input_dict["WFLOW_DESIGN"]["skip_emc"] = True      
#We can skip the effective mass calculation.

input_dict["WFLOW_DESIGN"]["skip_conv"] = True     
#We can skip the convergence test.

input_dict["WFLOW_DESIGN"]["skip_wannier"] = True  
#We can skip the wannier interpolation.

input_dict["WFLOW_DESIGN"]["scgw"] = False         
#We want to perform G0W0 calculation.

input_dict["WFLOW_DESIGN"]["skip_bse"] = False     
#We want to perform BSE calculation.


In [9]:
#Now let's create the new workflow
wf_gwbse=create_wfs(structure, input_dict)

-------------------------------------------
material:  Si_test
Information for efficient parallelization
You have  4 occupied bands
You have  56 kpoints
You have  [[11, 11, 11]] k-grid

KPAR= 8
reciprocal_density= 250
BSE calculation will include bands in the energy window (eV)= 3.0
-------------------------------------------


In [10]:
#Load the Fireworks Launchpad 
lpad = LaunchPad.auto_load() 

In [11]:
#Add the workflow you just created
lpad.add_wf(wf_gwbse)

2022-08-04 15:20:43,924 INFO Added a workflow. id_map: {-10: 7, -9: 8, -8: 9, -7: 10}


{-10: 7, -9: 8, -8: 9, -7: 10}

In [12]:
#Check the Fireworks loaded to the Launchpad
lp='lpad get_fws'
!{lp}

/home/tatha/anaconda3/envs/cms/lib/python3.6/site-packages/pymatgen/io/vasp/sets.py:447: BadInputSetWarning:

Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.

[
    {
        "fw_id": 1,
        "created_on": "2022-08-04T09:50:01.287246",
        "updated_on": "2022-08-04T09:50:01.287246",
        "name": "mp-661-WANNIER",
        "state": "WAITING"
    },
    {
        "fw_id": 2,
        "created_on": "2022-08-04T09:50:01.287228",
        "updated_on": "2022-08-04T09:50:01.287229",
        "name": "mp-661-GW",
        "state": "WAITING"
    },
    {
        "fw_id": 3,
        "created_on": "2022-08-04T09:50:01.287157",
        "updated_on": "2022-08-04T09:50:01.287158",
        "name": "mp-661-CONV",
        "state": "WAITING"
   

As you can see in addition to the 6 fireworks that was already there in the Launchpad,
now there are 4 more Fireworks with prefix 'Si_test'. Also we should note that the BSE 
Firework is in a 'PAUSED' state by default. This is because, the k-point parallelization 
is not yet implemented for BSE calculation in VASP. Therefore, to make the number of bands 
in the WAVEDER file a multiple of number of processor used (requirement for VASP, otherwise
it will change the NBANDS value) we need to run the BSE workflow on NPROCS=NPROCS'/KPAR, 
where NPROCS' is the number of processors used for the earlier Fireworks. Now that we have 
all the Fireworks in the Launchpad we can run them and use the postprocessing scripts to analyze
the results once they are 'COMPLETED'